In [36]:
from scraper import (
    fetch_html_selenium,
    save_raw_data,
    save_formatted_data,
    calculate_price,
    html_to_markdown_with_readability,
    create_dynamic_listing_model,
    create_listings_container_model,
    scrape_url,
    setup_selenium,
    generate_unique_folder_name
)
from assets import USER_AGENTS,PRICING,HEADLESS_OPTIONS,SYSTEM_MESSAGE,USER_MESSAGE,LLAMA_MODEL_FULLNAME,GROQ_LLAMA_MODEL_FULLNAME,HEADLESS_OPTIONS_DOCKER

import os
import google.generativeai as genai
from api_management import get_api_key
from dotenv import load_dotenv
load_dotenv()

True

In [41]:
def format_data1(data, DynamicListingsContainer, DynamicListingModel, selected_model):
    genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))
    model = genai.GenerativeModel(selected_model,
                generation_config={
                    "response_mime_type": "application/json",
                    "response_schema": DynamicListingsContainer
                })
    prompt = SYSTEM_MESSAGE + "\n" + USER_MESSAGE + data
    # Count input tokens using Gemini's method
    input_tokens = model.count_tokens(prompt)
    completion = model.generate_content(prompt)
    # Extract token counts from usage_metadata
    usage_metadata = completion.usage_metadata
    token_counts = {
            "input_tokens": usage_metadata.prompt_token_count,
            "output_tokens": usage_metadata.candidates_token_count
        }
    return completion.text, token_counts
    

In [28]:
url = 'https://sg.linkedin.com/company/sing-fuels'
field_names = [
    "Post Description",
    "Post Time",
    "Hashtags"
]
model_selection = "gemini-1.5-flash"

In [14]:
driver = setup_selenium(attended_mode=True)
driver.get(url)
output_folder = os.path.join('test_output', generate_unique_folder_name(url))
os.makedirs(output_folder, exist_ok=True)
raw_html = fetch_html_selenium(url, attended_mode=True, driver=driver)
markdown = html_to_markdown_with_readability(raw_html)
save_raw_data(markdown, output_folder, f'rawData_1.md')

Raw data saved to test_output\sg_linkedin_com_2025_01_23__22_54_29\rawData_1.md


'test_output\\sg_linkedin_com_2025_01_23__22_54_29\\rawData_1.md'

In [42]:
current_url = driver.current_url
DynamicListingModel = create_dynamic_listing_model(field_names)
DynamicListingsContainer = create_listings_container_model(DynamicListingModel)
formatted_data, token_counts = format_data1(
                    markdown, DynamicListingsContainer, DynamicListingModel, model_selection
                )

In [44]:
save_formatted_data(formatted_data, output_folder, f'sorted_data_1.json', f'sorted_data_1.xlsx')

Formatted data saved to JSON at test_output\sg_linkedin_com_2025_01_23__22_54_29\sorted_data_1.json
DataFrame created successfully.
Formatted data saved to Excel at test_output\sg_linkedin_com_2025_01_23__22_54_29\sorted_data_1.xlsx


,Hashtags,Post Description,Post Time
0,#sustainability#marinefuels#shipping#energytra...,“The marine transition is a trillion-dollar di...,1mo
1,#NewYear2025#HappyNewYear,Sing Fuels wishes you a joyful and prosperous ...,3w
2,#HolidayWishes#HappyHolidays#HolidaySeason#Chr...,"This holiday season, we wish you are surrounde...",4w
3,#FuelingSuccess#SingFuels,Meet the leaders that are#FuelingSuccessat Sin...,1mo
4,#FuelingSuccess#LeadershipAtSingFuels#Leadersh...,Meet the leaders that are#FuelingSuccessat Sin...,1mo
5,#teamappreciation#teamwork#energyservices#sing...,This week started on an incredible note with o...,1mo
6,#E50Awards#AwardNight#Internationalisation#Goi...,Special news! ✨ 🎉 We are thrilled to announce ...,1mo
7,#SingFuels7InARow#E50Award#E50Singapore#Singap...,We have big news! 🎉 Last night at the Enterpri...,1mo
8,#Diwali#SingFuels,May this festive season fill your lives with l...,2mo
9,#SingFuels#Anniversary#12YearsStrong#TeamSingF...,Celebrating 12 Years of Growth and Success 🎉 T...,3mo


In [20]:
field_definitions = {field: (str, ...) for field in field_names}
field_definitions

{'Post Description': (str, Ellipsis),
 'Post Time': (str, Ellipsis),
 'Hashtags': (str, Ellipsis)}

In [43]:
formatted_data

'{"listings": [{"Hashtags": "#sustainability#marinefuels#shipping#energytransition#ESG", "Post Description": "“The marine transition is a trillion-dollar dilemma for the next decade” andSing Fuels is poised to lead this transformative journey for the marineindustry. Our CEO, Vikash Dhanuka features in an exclusive interview with TheBusiness Times, where he shares insightsinto Sing Fuels’ mission to drive sustainability in the shipping industry. This vision is inspired by the ambitious targets set by the InternationalMaritime Organization: a 20% reduction in shipping emissions by 2030 and achieving net-zero emissions by 2050. Read thearticle authored by Paige Lim to discover how Sing Fuels is embracing bold changes and evolving into a pioneer of energy-transition.", "Post Time": "1mo"}, {"Hashtags": "#NewYear2025#HappyNewYear", "Post Description": "Sing Fuels wishes you a joyful and prosperous New Year ✨🎉 As we anchor-in on2025, we are ready to plunge into a successful year and create b